In [1]:
import numpy as np

#All possible partitions for our width 7 nanotube
all_partitions = SetPartitions(range(7))
filtered_partitions = [partition for partition in all_partitions if all(len(subset) <= 2 for subset in partition)]
n=0

#3 different sets of partitions for the 3 different cases
all_type2_partitions = [x for x in filtered_partitions if len(x) == 6]
partition4 = [x for x in filtered_partitions if len(x) == 5 and x.is_noncrossing()]
partition6 = [x for x in filtered_partitions if len(x) == 4 and x.is_noncrossing()]
        

#Valid partitions
valid_type2_pairs = [{0, 1}, {1, 2}, {2, 3}, {3, 4}, {4, 5}, {5, 6}, {6, 0}]
def is_valid_pair(s):
    for subset in s:
        if len(subset) == 2 and subset not in valid_type2_pairs:
            return False
    return True
partition2 = [s for s in all_type2_partitions if is_valid_pair(s)]

#Tiles
G = Graph()
for i in range(14):
    G.add_vertex(i)
    
#Visualise tiles
pos = {}
spacing = 1
for i in range(7):
    pos[i] = (0, -i * spacing)
    pos[i+7] = (2.5, -i * spacing)

G.add_edges([(0,7),(0,13)])
G.add_edges([(1,7),(1,8)])
G.add_edges([(2,9),(2,8)])
G.add_edges([(3,9),(3,10)])
G.add_edges([(4,11),(4,10)])
G.add_edges([(5,11),(5,12)])
G.add_edges([(6,13),(6,12)])


edges = G.edges(labels=False)
edge_subsets = Subsets(edges)
edge_subsets

#Tiles for 2 bridges
edge_subsets_12 = Subsets(edges,12)
edge_subsets_12

len(edge_subsets_12)


#Tiles for 4 bridges 
edge_subsets_10 = Subsets(edges,10)
edge_subsets_10

len(edge_subsets_10)

#Tiles for 6 bridges 
edge_subsets_8 = Subsets(edges,8)
edge_subsets_8

#Valid tiles - Can't be connected
disconnected_subsets_12 = []
for i in edge_subsets_12:
    H = Graph()
    H.add_edges(i)
    if H.is_connected() == False:
        disconnected_subsets_12.append(i)
    else:
        continue

disconnected_subsets_10 = []
for j in edge_subsets_10:
    H = Graph()
    H.add_edges(j)
    if H.is_connected() == False:
        disconnected_subsets_10.append(j)
    else:
        continue

        
disconnected_subsets_8 = []
for k in edge_subsets_8:
    H = Graph()
    H.add_edges(k)
    if H.is_connected() == False:
        disconnected_subsets_8.append(k)
    else:
        continue

#Valid subsets of length 12 (2 bridges)
valid_subsets_12 = []
for i in disconnected_subsets_12:
    H = Graph()
    H.add_edges(i)
    leaves = [v for v in H.vertices() if H.degree(v) == 1]
    lower = [leaf for leaf in leaves if leaf <= 6]
    upper = [leaf for leaf in leaves if leaf >= 7 ]
    if len(lower) == 2:
        #print(leaves)
        if H.distance(lower[0],lower[1]) == + Infinity:
            valid_subsets_12.append(i)
        
#valid subsets of length 10 (4 bridges)
valid_subsets_10 = []
for i in disconnected_subsets_10:
    H = Graph()
    H.add_edges(i)
    leaves = [v for v in H.vertices() if H.degree(v) == 1]
    lower = [leaf for leaf in leaves if leaf <= 6]
    upper = [leaf for leaf in leaves if leaf >= 7]
    if len(lower) == 4 and len(upper)== 4:
        #print(leaves)
        valid_subsets_10.append(i)
        

#Valid subsets of length 8 (6 bridges)
valid_subsets_8 = []
for i in disconnected_subsets_8:
    H = Graph()
    H.add_edges(i)
    leaves = [v for v in H.vertices() if H.degree(v) == 1]
    lower = [leaf for leaf in leaves if leaf <= 6]
    upper = [leaf for leaf in leaves if leaf >= 7]
    if len(lower)==6 and len(upper)==6:
        valid_subsets_8.append(i)


In [2]:
#ordering partitions
partition4ordered = []

def rotateP(ptn,k):
    rptn = []
    for c in ptn:
        rptn.append({(x+k)%7 for x in c})
    return SetPartition(rptn)

for i in range(len(partition4)):
    if  partition4[i] not in partition4ordered:
        for k in range(7):
            np = rotateP(partition4[i],k)
            partition4ordered.append(np)

partition6ordered = []

for i in range(len(partition6)):
    if  partition6[i] not in partition6ordered:
        for k in range(7):
            np = rotateP(partition6[i],k)
            partition6ordered.append(np)

In [6]:
import numpy as np
#Turning the partitions into dictionaries
partition2dict = {i: j for i, j in enumerate(partition2)}
partition6dict = {i: j for i, j in enumerate(partition6ordered)}
partition4dict = {i: j for i, j in enumerate(partition4ordered)}

#Initialising transfer matrices
transfermatrix2 = np.zeros((len(partition2), len(partition2)), dtype=object)
transfermatrix4 = np.zeros((len(partition4dict), len(partition4dict)), dtype=object)
transfermatrix6 = np.zeros((len(partition6dict), len(partition6dict)), dtype=object)

In [8]:
#Creating the transfer matrix for type 2 Hamilton cycles
for i in partition2dict.values():
    key1 = next(key for key, value in partition2dict.items() if value == i)
    for j in valid_subsets_12:
        H = Graph()
        H.add_edges(j)
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        inputvertices = [leaf for leaf in leaves if leaf <= 6]
        outputvertices = [leaf for leaf in leaves if leaf >= 7]
        if frozenset(inputvertices) == next(subset for subset in i if len(subset) == 2):
            outputcheck = [element - 7 for element in outputvertices]
            key2 = next(key for key, value in partition2dict.items() if frozenset(outputcheck) in value)
            transfermatrix2[key1][key2] += 1
        else:
            continue

In [9]:
#Defining functions to help create the type 6 and type 4 transfer matrices

def set_match(set1, inputvertices):
    check = [item for subset in set1 for item in subset if len(subset) == 2]
    check.sort()
    inputvertices.sort()
    return check == inputvertices

def find_output(G):
    if len(G.connected_components())==2:
        connected_components = G.connected_components()
        leaf_pairs = []
        for component in connected_components:
            leaves = [v for v in component if G.degree(v) == 1]
            if len(leaves) == 2:
                modified_leaves = tuple(sorted(leaf - 7 for leaf in leaves))
                leaf_pairs.append(modified_leaves)
        return leaf_pairs

    else:
        return False
                
                

def output_partition(leaf_pairs, partition4dict):
    required_matches = set(leaf_pairs)
    #matching_keys = []
    for key, partition in partition4dict.items():
        partition_sets = [tuple(sorted(pair)) for pair in partition]  
        if required_matches.issubset(partition_sets):
            #matching_keys.append(key)
            return key

In [11]:
#Constructing the type 4 transfer matrix
validis = []
for key1, i in partition4dict.items():
    for j in valid_subsets_10:
        H = Graph()
        H.add_edges(j)
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        inputvertices = [leaf for leaf in leaves if leaf <= 6]
        if set_match(i, inputvertices):
            #print("Matching set found: ", i, inputvertices)
            for k in i:
                if len(k) == 2:
                    H.add_edge(list(k)[0], list(k)[1])
            output = find_output(H)
            if output != False:
                validis.append(i)
                key2 = output_partition(output, partition4dict)
                if j not in validis:
                    validis.append(i)
                transfermatrix4[key1][key2]+=1
                print(key1, key2, j)

0 63 {(6, 12), (0, 7), (4, 10), (5, 11), (1, 8), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
0 35 {(6, 12), (4, 10), (0, 7), (5, 11), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12)}
0 36 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
1 37 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 7), (6, 13), (3, 9), (5, 12), (2, 8)}
1 64 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (6, 13), (3, 9), (5, 12)}
1 36 {(6, 12), (0, 7), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12)}
2 38 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (1, 7), (6, 13), (3, 9), (2, 8)}
2 65 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13)}
2 37 {(6, 12), (0, 7), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (4, 11), (5, 12)}
3 39 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (1, 7), (3, 9), (2, 8)}
3 66 {(6, 12), (0, 7), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (4, 11

22 28 {(0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (6, 13), (3, 9), (4, 11), (5, 12)}
22 50 {(0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12)}
23 9 {(6, 12), (0, 7), (4, 10), (5, 11), (1, 8), (1, 7), (6, 13), (3, 9), (5, 12), (2, 8)}
23 29 {(6, 12), (0, 7), (4, 10), (5, 11), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (5, 12)}
23 51 {(6, 12), (0, 7), (5, 11), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (4, 11), (5, 12)}
23 52 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (1, 7), (6, 13), (3, 9), (5, 12), (2, 8)}
23 7 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (5, 12)}
24 53 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (2, 9), (1, 7), (6, 13), (3, 9), (2, 8)}
24 8 {(6, 12), (0, 7), (5, 11), (0, 13), (3, 10), (2, 9), (1, 7), (6, 13), (4, 11), (2, 8)}
24 10 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (6, 13), (3, 9), (2, 8)}
24 30 {(6, 12), (0, 7), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (6, 13)

40 3 {(0, 7), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
40 4 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (3, 9), (2, 8)}
40 26 {(6, 12), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (3, 9), (4, 11), (2, 8)}
40 46 {(0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
41 47 {(0, 7), (4, 10), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
41 4 {(0, 7), (4, 10), (1, 8), (3, 10), (2, 9), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
41 5 {(6, 12), (4, 10), (5, 11), (0, 13), (3, 10), (2, 9), (1, 7), (3, 9), (4, 11), (2, 8)}
41 27 {(6, 12), (4, 10), (0, 13), (3, 10), (2, 9), (1, 7), (3, 9), (4, 11), (5, 12), (2, 8)}
42 55 {(0, 7), (4, 10), (5, 11), (1, 8), (2, 9), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
42 12 {(0, 7), (4, 10), (5, 11), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12), (2, 8)}
42 13 {(6, 12), (4, 10), (5, 11), (0, 13), (2, 9), (1, 7), (3, 9), (4, 11), (5, 12

58 54 {(6, 12), (0, 7), (4, 10), (5, 11), (1, 8), (3, 10), (1, 7), (6, 13), (3, 9), (2, 8)}
58 9 {(6, 12), (0, 7), (1, 8), (3, 10), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12), (2, 8)}
58 33 {(6, 12), (0, 7), (5, 11), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11)}
58 31 {(6, 12), (0, 7), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11), (5, 12)}
58 32 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (1, 7), (6, 13), (3, 9), (2, 8)}
58 30 {(6, 12), (5, 11), (0, 13), (1, 8), (3, 10), (1, 7), (6, 13), (3, 9), (4, 11), (2, 8)}
58 28 {(6, 12), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (3, 9), (4, 11)}
59 33 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (2, 9), (1, 7), (3, 9), (4, 11), (2, 8)}
59 31 {(6, 12), (0, 7), (4, 10), (0, 13), (2, 9), (1, 7), (3, 9), (4, 11), (5, 12), (2, 8)}
59 29 {(6, 12), (0, 7), (4, 10), (0, 13), (3, 10), (2, 9), (1, 7), (4, 11), (5, 12), (2, 8)}
59 55 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (3, 9), (4

In [14]:
#Constructing the type 6 transfer matrix
def find_output6(G):
    if len(G.connected_components()) == 3:
        connected_components = G.connected_components()
        leaf_pairs = []
        for component in connected_components:
            leaves = [v for v in component if G.degree(v) == 1]
            if len(leaves) == 2:
                modified_leaves = tuple(sorted(leaf - 7 for leaf in leaves))
                leaf_pairs.append(modified_leaves)
        return leaf_pairs
    else:
        return False

    
for key1, i in partition6dict.items():
    for j in valid_subsets_8:
        H = Graph()
        H.add_edges(j)
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        inputvertices = [leaf for leaf in leaves if leaf <= 6]
        if set_match(i, inputvertices):
            #print("Matching set found: ", i, inputvertices)
            for k in i:
                if len(k) == 2:
                    H.add_edge(list(k)[0], list(k)[1])
            output = find_output6(H)
            if output != False:
                key2 = output_partition(output, partition6dict)
                transfermatrix6[key1][key2]+=1
                print(key1, key2, j)                    

0 10 {(6, 12), (0, 7), (4, 10), (5, 11), (1, 8), (6, 13), (3, 9), (2, 8)}
0 12 {(6, 12), (4, 10), (0, 7), (5, 11), (1, 8), (3, 10), (2, 9), (6, 13)}
0 7 {(6, 12), (0, 7), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12)}
0 8 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 7), (6, 13), (3, 9), (2, 8)}
1 9 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 7), (3, 9), (2, 8)}
1 11 {(6, 12), (0, 7), (4, 10), (5, 11), (0, 13), (1, 8), (2, 9), (3, 9)}
1 13 {(6, 12), (0, 7), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (4, 11)}
1 8 {(0, 7), (0, 13), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12)}
2 9 {(0, 7), (1, 8), (3, 10), (2, 9), (1, 7), (6, 13), (4, 11), (5, 12)}
2 10 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (1, 7), (3, 9), (2, 8)}
2 12 {(6, 12), (4, 10), (5, 11), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7)}
2 7 {(6, 12), (0, 13), (1, 8), (3, 10), (2, 9), (1, 7), (4, 11), (5, 12)}
3 10 {(0, 7), (1, 8), (3, 10), (2, 9), (6, 13), (4, 11), (5, 12), (2, 8)}
3 11 {(6, 12), (4, 10), (5, 11), (0, 

In [15]:
#Matrix types
transfermatrixmatrix4 = matrix(transfermatrix4)
transfermatrixmatrix6 = matrix(transfermatrix6)

In [19]:
#View the matrix in sections of orbits
for i in range(10):
    for j in range(10):
        print(i,j)
        print(transfermatrixmatrix4.matrix_from_rows_and_columns(range(7*i, 7*(i+1)),range(7*j,7*(j+1))))

0 0
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 1
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 2
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 3
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 4
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 5
[1 1 0 0 0 0 0]
[0 1 1 0 0 0 0]
[0 0 1 1 0 0 0]
[0 0 0 1 1 0 0]
[0 0 0 0 1 1 0]
[0 0 0 0 0 1 1]
[1 0 0 0 0 0 1]
0 6
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 7
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 8
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0

In [18]:
#View matrix in sections of orbits
for i in range(5):
    for j in range(5):
        print(i,j)
        print(transfermatrixmatrix6.matrix_from_rows_and_columns(range(7*i, 7*(i+1)),range(7*j,7*(j+1))))   

0 0
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 1
[1 1 0 1 0 1 0]
[0 1 1 0 1 0 1]
[1 0 1 1 0 1 0]
[0 1 0 1 1 0 1]
[1 0 1 0 1 1 0]
[0 1 0 1 0 1 1]
[1 0 1 0 1 0 1]
0 2
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 3
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
0 4
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
1 0
[0 0 0 0 0 1 1]
[1 0 0 0 0 0 1]
[1 1 0 0 0 0 0]
[0 1 1 0 0 0 0]
[0 0 1 1 0 0 0]
[0 0 0 1 1 0 0]
[0 0 0 0 1 1 0]
1 1
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
1 2
[0 0 0 0 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
1 3
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0